In [1]:
import goatools
import seaborn as sns
import pandas as pd 
import pickle

In [2]:
# Necessary Imports for running a full GO Enrichment Analysis --> 2.4

from goatools.obo_parser import GODag

from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

# Load Data

In [3]:
df_all_merged_kmeans_data_file = "../data/go_terms/df_all_merged_kmeans_data.p"
df_all_merged_birch_data_file = "../data/go_terms/df_all_merged_birch_data.p"

df_all_merged_kmeans_data = pickle.load(open(df_all_merged_kmeans_data_file, "rb"))
df_all_merged_birch_data = pickle.load(open(df_all_merged_birch_data_file, "rb"))

In [4]:
kmeans_19_file = "../data/cluster_by_loci/kmeans, param1 = 19, param2 = nan, param3 = nan"

df_kmeans_labels = pd.read_csv(kmeans_19_file).fillna('')

In [5]:
# sourced from data derived from Blast2Go Database

dict_blast2go_file = "../data/go_terms/dict_blast2go_loci_to_go.pickle"


loci_to_go = pickle.load(open(dict_blast2go_file, 'rb'))
loci_to_go

{'EQU24_RS00005': ['GO:0003688',
  'GO:0005524',
  'GO:0005737',
  'GO:0006270',
  'GO:0006275'],
 'EQU24_RS00010': ['GO:0003677',
  'GO:0003887',
  'GO:0005737',
  'GO:0006260',
  'GO:0008408',
  'GO:0009360',
  'GO:0071897',
  'GO:0090305'],
 'EQU24_RS00015': ['GO:0003697',
  'GO:0005524',
  'GO:0005737',
  'GO:0006260',
  'GO:0006281',
  'GO:0009432'],
 'EQU24_RS00020': ['GO:0003677',
  'GO:0003918',
  'GO:0005524',
  'GO:0005694',
  'GO:0005737',
  'GO:0006261',
  'GO:0006265',
  'GO:0046872'],
 'EQU24_RS00040': ['GO:0015562', 'GO:0055085'],
 'EQU24_RS00045': ['GO:0016020', 'GO:0022857', 'GO:0055085'],
 'EQU24_RS00050': ['GO:0004601',
  'GO:0008324',
  'GO:0016021',
  'GO:0098655',
  'GO:0098869'],
 'EQU24_RS00060': ['GO:0005515', 'GO:0016021', 'GO:0016740'],
 'EQU24_RS00065': ['GO:0016021'],
 'EQU24_RS00070': ['GO:0005887', 'GO:0008320', 'GO:0033281', 'GO:0043953'],
 'EQU24_RS00075': ['GO:0005887', 'GO:0008320', 'GO:0033281', 'GO:0043953'],
 'EQU24_RS00080': ['GO:0005506', 'GO:001

## Visualize Imported Dataframes

- Data represents each locus, its expression under the different conditions, go_terms, and metadata regarding the locus itself

In [6]:
df_all_merged_kmeans_data

,locus_tag,cluster_id,LanzaTech,MeOH,NO3_lowO2_slow_growth,NoCu,NoLanthanum,WT_control,WithLanthanum,aa3_KO,...,unknown,gene,product,start_coord,end_coord,length,translation,group,go_terms,desc_string
0,EQU24_RS00005,12,-0.515808,-0.012058,-0.480048,-0.163349,-0.648447,-0.037469,-0.491013,-0.410117,...,-0.620522,dnaA,chromosomal replication initiator protein DnaA,0,1317,1318,MSALWNNCLAKLENEISSSEFSTWIRPLQAIETDGQIKLLAPNRFV...,else,"[GO:0003688, GO:0005524, GO:0005737, GO:000627...",EQU24_RS00005|dnaA|chromosomal replication ini...
1,EQU24_RS00010,4,0.480568,-0.659004,-0.218643,0.075821,-0.485571,-0.114821,-0.462526,0.094641,...,-0.798028,,DNA polymerase III subunit beta,1502,2603,1102,MKYIINREQLLVPLQQIVSVIEKRQTMPILSNVLMVFRENTLVMTG...,else,"[GO:0003677, GO:0003887, GO:0005737, GO:000626...",EQU24_RS00010||DNA polymerase III subunit beta
2,EQU24_RS00015,17,-1.072637,0.327816,-0.461533,-0.212764,-0.583017,-0.648530,-0.633792,-0.917224,...,-0.753895,recF,DNA replication/repair protein RecF,3060,4140,1081,MSLQKLDIFNVRNIRQASLQPSPGLNLIYGANASGKSSVLEAIFIL...,else,"[GO:0003697, GO:0005524, GO:0005737, GO:000626...",EQU24_RS00015|recF|DNA replication/repair prot...
3,EQU24_RS00020,4,0.723618,-1.032390,0.072132,-0.253222,-0.352283,0.183488,-0.292974,0.144541,...,-0.532597,gyrB,DNA topoisomerase (ATP-hydrolyzing) subunit B,4185,6600,2416,MSENIKQYDSTNIQVLKGLDAVRKRPGMYIGDTDDGTGLHHMVFEV...,else,"[GO:0003677, GO:0003918, GO:0005524, GO:000569...",EQU24_RS00020|gyrB|DNA topoisomerase (ATP-hydr...
4,EQU24_RS00025,4,0.424262,-0.479325,0.028098,0.116538,-0.261659,0.082847,-0.454102,-0.051229,...,-0.011128,,hypothetical protein,6825,7062,238,VKTTKYFLTTRMRPDREIIKDEWIQYVVRFPENEHIQFDGRIRRWA...,else,[],EQU24_RS00025||hypothetical protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208,EQU24_RS22135,9,-0.445389,-0.627550,-1.170665,-0.396450,-1.877416,-0.819438,-1.715711,-0.966683,...,-0.784262,mnmE,tRNA uridine-5-carboxymethylaminomethyl(34) sy...,4994941,4996288,1348,VDIGTNDTIAAIATPPGNGGVGIVRISGPAVSIIAAKLTDRPLPPR...,else,"[GO:0003924, GO:0005515, GO:0005525, GO:000573...",EQU24_RS22135|mnmE|tRNA uridine-5-carboxymethy...
4209,EQU24_RS22140,9,0.309593,-1.327737,-1.065740,-0.706776,-1.812366,-0.818297,-1.749368,-0.654022,...,-1.159766,yidC,membrane protein insertase YidC,4996295,4997993,1699,MDNIRFILVVTFAMLLYMLYDAWQIDYGPKREISVAQQMPQDLKED...,else,"[GO:0005886, GO:0015031, GO:0016021, GO:003297...",EQU24_RS22140|yidC|membrane protein insertase ...
4210,EQU24_RS22145,0,-1.028264,-0.395724,-1.265702,-0.657315,-2.492508,-0.803695,-2.140667,-0.926349,...,-1.151259,yidD,membrane protein insertion efficiency factor YidD,4997998,4998220,223,MRVLLIAIIKLYQYFISPLLGKNCRFYPSCSCYALEALHKHGAAQG...,else,[GO:0005886],EQU24_RS22145|yidD|membrane protein insertion ...
4211,EQU24_RS22150,0,-0.670633,-0.279756,-1.404703,-0.775605,-2.384838,-1.209018,-2.238072,-1.143556,...,-1.159678,rnpA,ribonuclease P protein component,4998201,4998570,370,LTNKVFSFPPQLRLRKPSEYKKVFTGPVKSSDAYFTLLAVRNELDH...,else,"[GO:0000049, GO:0001682, GO:0004526, GO:0090502]",EQU24_RS22150|rnpA|ribonuclease P protein comp...


In [7]:
df_all_merged_birch_data['cluster_id']

0       14
1       14
2        3
3       14
4       21
        ..
4208     9
4209     9
4210     9
4211     9
4212     9
Name: cluster_id, Length: 4213, dtype: int64

In [8]:
# For each cluster in this clustering, a set of unique go terms will be developed
def get_go_terms_for_clustering(df_clustering):

    clusters = range(max(df_clustering['cluster_id']) + 1)
    cluster_to_go_term = {}

    for cluster_id in clusters:

        x = df_clustering[df_clustering['cluster_id']== cluster_id]
        go_terms_in_cluster = []
        for group in x['go_terms']: 
            for single in group: go_terms_in_cluster.append(single) 
        go_terms_in_cluster = set(go_terms_in_cluster)
        #print(go_terms_in_cluster)

        cluster_to_go_term[cluster_id] = go_terms_in_cluster

    return cluster_to_go_term   
    

In [9]:
birch_cluster_2_goterms = get_go_terms_for_clustering(df_all_merged_birch_data)
birch_cluster_2_goterms

{0: {'GO:0000272',
  'GO:0003677',
  'GO:0003700',
  'GO:0004190',
  'GO:0004252',
  'GO:0004601',
  'GO:0005509',
  'GO:0005524',
  'GO:0006508',
  'GO:0007155',
  'GO:0009055',
  'GO:0009306',
  'GO:0016020',
  'GO:0016021',
  'GO:0016887',
  'GO:0016987',
  'GO:0020037',
  'GO:0022900',
  'GO:0035438',
  'GO:0046872',
  'GO:0098869',
  'GO:2000142'},
 1: {'GO:0003824',
  'GO:0004410',
  'GO:0004792',
  'GO:0005524',
  'GO:0005737',
  'GO:0005886',
  'GO:0006535',
  'GO:0009001',
  'GO:0009055',
  'GO:0009108',
  'GO:0009399',
  'GO:0010181',
  'GO:0016021',
  'GO:0016163',
  'GO:0016612',
  'GO:0016787',
  'GO:0018697',
  'GO:0019752',
  'GO:0022900',
  'GO:0030151',
  'GO:0046872',
  'GO:0051188',
  'GO:0051536',
  'GO:0051537',
  'GO:0051539',
  'GO:0055085',
  'GO:0055114'},
 2: {'GO:0003677',
  'GO:0004252',
  'GO:0004803',
  'GO:0006260',
  'GO:0006281',
  'GO:0006313',
  'GO:0006355',
  'GO:0006508',
  'GO:0009432',
  'GO:0015074',
  'GO:0016020',
  'GO:0016021',
  'GO:0016779

In [10]:
kmeans_cluster_2_goterms = get_go_terms_for_clustering(df_all_merged_kmeans_data)
kmeans_cluster_2_goterms

{0: {'GO:0000027',
  'GO:0000036',
  'GO:0000049',
  'GO:0000155',
  'GO:0000160',
  'GO:0000166',
  'GO:0000175',
  'GO:0000287',
  'GO:0000413',
  'GO:0001123',
  'GO:0001522',
  'GO:0001682',
  'GO:0002935',
  'GO:0003674',
  'GO:0003676',
  'GO:0003677',
  'GO:0003700',
  'GO:0003723',
  'GO:0003735',
  'GO:0003746',
  'GO:0003755',
  'GO:0003849',
  'GO:0003852',
  'GO:0003883',
  'GO:0003924',
  'GO:0003992',
  'GO:0004019',
  'GO:0004055',
  'GO:0004222',
  'GO:0004316',
  'GO:0004386',
  'GO:0004397',
  'GO:0004413',
  'GO:0004519',
  'GO:0004521',
  'GO:0004526',
  'GO:0004595',
  'GO:0004638',
  'GO:0004749',
  'GO:0004810',
  'GO:0004824',
  'GO:0004825',
  'GO:0004826',
  'GO:0004829',
  'GO:0004851',
  'GO:0005515',
  'GO:0005524',
  'GO:0005525',
  'GO:0005575',
  'GO:0005737',
  'GO:0005840',
  'GO:0005886',
  'GO:0006015',
  'GO:0006164',
  'GO:0006189',
  'GO:0006281',
  'GO:0006310',
  'GO:0006355',
  'GO:0006364',
  'GO:0006400',
  'GO:0006412',
  'GO:0006414',
  'GO

# Applying GOATOOLS

- Working loosely from the goatools documentation example of using their library for exploration of gene ontology data for a given set of GO Terms

## Practice with Example

In [11]:
from goatools.base import get_godag
godag = get_godag("go-basic.obo")

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2022-01-13) 47,158 GO Terms


In [12]:
type(godag)

goatools.obo_parser.GODag

In [13]:
from goatools.gosubdag.gosubdag import GoSubDag

In [14]:
DESC = 'methane'
NSPC = 'cellular_component'

def chooser(goterm):
    b_match = DESC in goterm.name
    
    b_leaf = not goterm.children
    
    b_nspc = goterm.namespace == NSPC
    
    return b_match and b_leaf and b_nspc

# Set of go_terms!
go_ids_selected = set(o.item_id for o in godag.values() if chooser(o))

print('{N} {desc} GO Terms'.format(N= len(go_ids_selected), desc = DESC))

2 methane GO Terms


In [15]:
gosubdag = GoSubDag(go_ids_selected, godag)

 GoSubDag:   2 sources in  11 GOs rcnt(True). 0 alt GO IDs
 GoSubDag: namedtuple fields: NS level depth GO alt GO_name dcnt D1 id
 GoSubDag: relationships: set()


In [16]:
go_id, go_term = max(gosubdag.go2obj.items(), key = lambda t: t[1].depth)
print(go_id, go_term.name)

GO:0044227 methane-oxidizing organelle


In [17]:
go_ids_chosen = go_term.get_all_parents()

print('{N} ancestors for {GO} "{name}"'.format(N = len(go_ids_chosen), GO = go_term.item_id, name = go_term.name))

6 ancestors for GO:0044227 "methane-oxidizing organelle"


In [18]:
go_ids_chosen.add(go_id)
nts = [gosubdag.go2nt[go] for go in go_ids_chosen]

In [19]:
fmt_str = '{I:2}) {NS} {GO:10} {dcnt:11}        D{depth:02}  {GO_name}'
print('IDX NS GO ID      Descendants Count Depth Name')
print('--- -- ---------- ----------------- ----- ------------------')
for idx, nt_go in enumerate(sorted(nts, key=lambda nt: nt.depth), 1):
    print(fmt_str.format(I=idx, **nt_go._asdict()))

IDX NS GO ID      Descendants Count Depth Name
--- -- ---------- ----------------- ----- ------------------
 1) CC GO:0005575        4182        D00  cellular_component
 2) CC GO:0110165        2036        D01  cellular anatomical entity
 3) CC GO:0043226         396        D02  organelle
 4) CC GO:0043227         241        D03  membrane-bounded organelle
 5) CC GO:0043229         346        D03  intracellular organelle
 6) CC GO:0043231         207        D04  intracellular membrane-bounded organelle
 7) CC GO:0044227           0        D05  methane-oxidizing organelle


## Practice with Real Sample

In [20]:
# SAMPLE => Loci: EQU24_RS22135

loci_of_interest = 'EQU24_RS22135'
loci_data_row = df_all_merged_birch_data[df_all_merged_birch_data['locus_tag'] == loci_of_interest]
go_terms_of_interest = loci_data_row['go_terms'].values[0]

go_terms_of_interest

['GO:0003924',
 'GO:0005515',
 'GO:0005525',
 'GO:0005737',
 'GO:0006400',
 'GO:0046872']

In [21]:
pd.set_option('display.max_colwidth', None)

loci_data_row['desc_string']

4208    EQU24_RS22135|mnmE|tRNA uridine-5-carboxymethylaminomethyl(34) synthesis GTPase MnmE
Name: desc_string, dtype: object

In [22]:
gosubdag = GoSubDag(go_terms_of_interest, godag)

 GoSubDag:   6 sources in  53 GOs rcnt(True). 0 alt GO IDs
 GoSubDag: namedtuple fields: NS level depth GO alt GO_name dcnt D1 id
 GoSubDag: relationships: set()


In [23]:
go_id, go_term = max(gosubdag.go2obj.items(), key = lambda t: t[1].depth)
print(go_id, go_term.name)

GO:0006400 tRNA modification


In [24]:
go_ids_chosen = go_term.get_all_parents()

print('{N} ancestors for {GO} "{name}"'.format(N = len(go_ids_chosen), GO = go_term.item_id, name = go_term.name))

22 ancestors for GO:0006400 "tRNA modification"


In [25]:
go_ids_chosen.add(go_id)
nts = [gosubdag.go2nt[go] for go in go_ids_chosen]

In [26]:
fmt_str = '{I:2}) {NS} {GO:10} {dcnt:11}        D{depth:02}  {GO_name}'
print('IDX NS GO ID      Descendants Count Depth Name')
print('--- -- ---------- ----------------- ----- ------------------')
for idx, nt_go in enumerate(sorted(nts, key=lambda nt: nt.depth), 1):
    print(fmt_str.format(I=idx, **nt_go._asdict()))

IDX NS GO ID      Descendants Count Depth Name
--- -- ---------- ----------------- ----- ------------------
 1) BP GO:0008150       28427        D00  biological_process
 2) BP GO:0009987       11907        D01  cellular process
 3) BP GO:0008152        6296        D01  metabolic process
 4) BP GO:0044238        3948        D02  primary metabolic process
 5) BP GO:0044237        5474        D02  cellular metabolic process
 6) BP GO:0071704        5965        D02  organic substance metabolic process
 7) BP GO:0006807        3737        D02  nitrogen compound metabolic process
 8) BP GO:0006725        2094        D03  cellular aromatic compound metabolic process
 9) BP GO:0046483        2025        D03  heterocycle metabolic process
10) BP GO:0034641        2038        D03  cellular nitrogen compound metabolic process
11) BP GO:1901360        2475        D03  organic cyclic compound metabolic process
12) BP GO:0043170        2279        D03  macromolecule metabolic process
13) BP GO:00061

## Practice: Using Cluster-Wide Data to Seek Enrichment

- Applying cluster 5 (almost all repressible copper loci and smo) from Kmeans Clustering

In [27]:
cluster_5 = kmeans_cluster_2_goterms[5]

In [28]:
gosubdag = GoSubDag(cluster_5, godag)

1 GO IDs NOT FOUND IN GO DAG: GO:0055114
 GoSubDag:  33 sources in 150 GOs rcnt(True). 0 alt GO IDs
 GoSubDag: namedtuple fields: NS level depth GO alt GO_name dcnt D1 id
 GoSubDag: relationships: set()


In [29]:
go_id, go_term = max(gosubdag.go2obj.items(), key = lambda t: t[1].depth)
print(go_id, go_term.name)

GO:2000142 regulation of DNA-templated transcription, initiation


In [30]:
go_ids_chosen = go_term.get_all_parents()

print('{N} ancestors for {GO} "{name}"'.format(N = len(go_ids_chosen), GO = go_term.item_id, name = go_term.name))

18 ancestors for GO:2000142 "regulation of DNA-templated transcription, initiation"


In [31]:
go_ids_chosen.add(go_id)
nts = [gosubdag.go2nt[go] for go in go_ids_chosen]

In [32]:
fmt_str = '{I:2}) {NS} {GO:10} {dcnt:11}        D{depth:02}  {GO_name}'
print('IDX NS GO ID      Descendants Count Depth Name')
print('--- -- ---------- ----------------- ----- ------------------')
for idx, nt_go in enumerate(sorted(nts, key=lambda nt: nt.depth), 1):
    print(fmt_str.format(I=idx, **nt_go._asdict()))

IDX NS GO ID      Descendants Count Depth Name
--- -- ---------- ----------------- ----- ------------------
 1) BP GO:0008150       28427        D00  biological_process
 2) BP GO:0065007       11852        D01  biological regulation
 3) BP GO:0050789       10773        D02  regulation of biological process
 4) BP GO:0050794        8103        D03  regulation of cellular process
 5) BP GO:0019222        3152        D03  regulation of metabolic process
 6) BP GO:0060255        2001        D04  regulation of macromolecule metabolic process
 7) BP GO:0031323        2443        D04  regulation of cellular metabolic process
 8) BP GO:0051171        1752        D04  regulation of nitrogen compound metabolic process
 9) BP GO:0009889        1186        D04  regulation of biosynthetic process
10) BP GO:0080090        1981        D04  regulation of primary metabolic process
11) BP GO:0010468         967        D05  regulation of gene expression
12) BP GO:0019219         726        D05  regulatio

## Application: Running a full Gene Ontology Enrichment Analysis (GOEA)

### Single Cluster Example

In [33]:
obodag = GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2022-01-13) 47,158 GO Terms


In [34]:
len(loci_to_go)

3411

In [35]:
test_dict = {"test": loci_to_go}

In [36]:
goeaobj = GOEnrichmentStudyNS(
        list(loci_to_go.keys()), # List of all labeled loci
        test_dict, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest


Load test Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
100%  3,411 of  3,411 population items found in association


<b>Observation</b>

It is noted that all of the loci have associated GO Terms AND this is observed since we are supplying only loci that have GO terms to this enrichment analysis. 

Question -- should we include loci that don't have go terms? would that mess up this analysis? How do we bridge that data in?

In [86]:
cluster_5_genes = df_all_merged_kmeans_data[df_all_merged_kmeans_data["cluster_id"] == 5]['locus_tag'].values

cluster_5_genes

array(['EQU24_RS00670', 'EQU24_RS00940', 'EQU24_RS01900', 'EQU24_RS05885',
       'EQU24_RS05890', 'EQU24_RS05895', 'EQU24_RS05900', 'EQU24_RS05905',
       'EQU24_RS05910', 'EQU24_RS05915', 'EQU24_RS05920', 'EQU24_RS05925',
       'EQU24_RS07675', 'EQU24_RS07680', 'EQU24_RS15800', 'EQU24_RS19450',
       'EQU24_RS19455', 'EQU24_RS19460', 'EQU24_RS19465', 'EQU24_RS19470',
       'EQU24_RS19475', 'EQU24_RS19480', 'EQU24_RS19485', 'EQU24_RS19490',
       'EQU24_RS19495', 'EQU24_RS19500', 'EQU24_RS19505', 'EQU24_RS19510',
       'EQU24_RS19515', 'EQU24_RS19520', 'EQU24_RS21000', 'EQU24_RS21005'],
      dtype=object)

In [87]:
goea_results_all = goeaobj.run_study(cluster_5_genes)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run test Gene Ontology Analysis: current study set of 32 IDs ...100%     26 of     26 study items found in association
 81%     26 of     32 study items found in population(3411)
Calculating 1,742 uncorrected p-values using fisher_scipy_stats
   1,742 GO terms are associated with  3,407 of  3,411 population items
      33 GO terms are associated with     26 of     32 study items
  METHOD fdr_bh:
       2 GO terms found significant (< 0.05=alpha) (  2 enriched +   0 purified): statsmodels fdr_bh
       5 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)


In [88]:
goea_results_sig

[GOEnrichmentRecord(GO:0006725), GOEnrichmentRecord(GO:0015049)]

In [89]:
goea_results_sig[0].goterm

GOTerm('GO:0006725'):
  id:GO:0006725
  item_id:GO:0006725
  name:cellular aromatic compound metabolic process
  namespace:biological_process
  _parents: 1 items
    GO:0044237
  parents: 1 items
    GO:0044237	level-02	depth-02	cellular metabolic process [biological_process]
  children: 46 items
  level:3
  depth:3
  is_obsolete:False
  alt_ids: 0 items

### Generate Full GO Term Dataset

In [90]:
def initialize_enrichment_analysis_params(): 
    
    obodag = GODag("go-basic.obo")
    
    loci2go_dict = {"loci2go": loci_to_go}
    
    goeaobj = GOEnrichmentStudyNS(
        list(loci_to_go.keys()), # List of all labeled loci
        loci2go_dict, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest
    
    return goeaobj
    

In [121]:
def generate_full_enrichment_data(clustering_data): 
    
    # Generate the object that does the statistical analysis for the enrichment analysis
    goeaobj_for_study = initialize_enrichment_analysis_params()
    
    # Total number of clusters in this method: 
    total_cluster_num = max(clustering_data["cluster_id"]) + 1
    
    # Generate dataframe to hold 
    
    enrichment_scores = []
    
    # Given a clustering method, loop through the clusters
    for cluster_num in range(0, total_cluster_num):
        
        # For each cluster's loci, check for enriched GO terms
        current_cluster_loci = clustering_data[clustering_data["cluster_id"] == cluster_num]['locus_tag'].values
        
        # Run enrichment analysis for cluster
        goea_current_cluster_all_loci = goeaobj_for_study.run_study(current_cluster_loci)
        goea_current_cluster_sig_loci = [r for r in goea_current_cluster_all_loci if r.p_fdr_bh < 0.05]
        
        enrichment_scores.append(goea_current_cluster_sig_loci)
        
    return enrichment_scores#pd.DataFrame(enrichment_scores).T
    

## kMeans Enrichment Analysis

In [128]:
df_kmeans_full_enrichment_data = generate_full_enrichment_data(df_all_merged_kmeans_data)

df_kmeans_full_enrichment_data

go-basic.obo: fmt(1.2) rel(2022-01-13) 47,158 GO Terms

Load loci2go Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
100%  3,411 of  3,411 population items found in association

Run loci2go Gene Ontology Analysis: current study set of 167 IDs ...100%    131 of    131 study items found in association
 78%    131 of    167 study items found in population(3411)
Calculating 1,742 uncorrected p-values using fisher_scipy_stats
   1,742 GO terms are associated with  3,407 of  3,411 population items
     209 GO terms are associated with    131 of    167 study items
  METHOD fdr_bh:
       8 GO terms found significant (< 0.05=alpha) (  8 enriched +   0 purified): statsmodels fdr_bh
      75 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run loci2go Gene Ontology Analysis: current study set of 763 IDs ...100%    563 of    563 study items found in association
 74%    56

   1,742 GO terms are associated with  3,407 of  3,411 population items
      21 GO terms are associated with     13 of     15 study items
  METHOD fdr_bh:
       1 GO terms found significant (< 0.05=alpha) (  1 enriched +   0 purified): statsmodels fdr_bh
       2 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run loci2go Gene Ontology Analysis: current study set of 54 IDs ...100%     32 of     32 study items found in association
 59%     32 of     54 study items found in population(3411)
Calculating 1,742 uncorrected p-values using fisher_scipy_stats
   1,742 GO terms are associated with  3,407 of  3,411 population items
      33 GO terms are associated with     32 of     54 study items
  METHOD fdr_bh:
       6 GO terms found significant (< 0.05=alpha) (  6 enriched +   0 purified): statsmodels fdr_bh
      19 study items associated with significant GO IDs (enriched)
       0 study items associated with s

[[GOEnrichmentRecord(GO:0009236),
  GOEnrichmentRecord(GO:0006412),
  GOEnrichmentRecord(GO:0006364),
  GOEnrichmentRecord(GO:0005840),
  GOEnrichmentRecord(GO:0005737),
  GOEnrichmentRecord(GO:0003735),
  GOEnrichmentRecord(GO:0000049),
  GOEnrichmentRecord(GO:0005524)],
 [GOEnrichmentRecord(GO:0000160), GOEnrichmentRecord(GO:0005737)],
 [],
 [GOEnrichmentRecord(GO:0006278),
  GOEnrichmentRecord(GO:0006313),
  GOEnrichmentRecord(GO:0003964),
  GOEnrichmentRecord(GO:0004803),
  GOEnrichmentRecord(GO:0004802)],
 [GOEnrichmentRecord(GO:0006814),
  GOEnrichmentRecord(GO:0005737),
  GOEnrichmentRecord(GO:0016655),
  GOEnrichmentRecord(GO:0030170),
  GOEnrichmentRecord(GO:0006313),
  GOEnrichmentRecord(GO:0004803),
  GOEnrichmentRecord(GO:0003735)],
 [GOEnrichmentRecord(GO:0006725), GOEnrichmentRecord(GO:0015049)],
 [GOEnrichmentRecord(GO:0006935),
  GOEnrichmentRecord(GO:0007165),
  GOEnrichmentRecord(GO:0000160),
  GOEnrichmentRecord(GO:0004888)],
 [GOEnrichmentRecord(GO:0071973),
  GOEnr

In [134]:
kmeans_summary_enrichment_results= []
for cluster in df_kmeans_full_enrichment_data:
    summary_terms = [[go_term.GO, go_term.name] for go_term in cluster]
    kmeans_summary_enrichment_results.append(summary_terms)
    
df_kmeans_summary_enrichment_analysis = pd.DataFrame(kmeans_summary_enrichment_results).T
pickle.dump(df_kmeans_summary_enrichment_analysis, open('../data/go_terms/df_kmeans_summary_enrichment_analysis.pickle', 'wb'))

df_kmeans_summary_enrichment_analysis

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,"[GO:0009236, cobalamin biosynthetic process]","[GO:0000160, phosphorelay signal transduction system]",None,"[GO:0006278, RNA-dependent DNA biosynthetic process]","[GO:0006814, sodium ion transport]","[GO:0006725, cellular aromatic compound metabolic process]","[GO:0006935, chemotaxis]","[GO:0071973, bacterial-type flagellum-dependent cell motility]","[GO:0009399, nitrogen fixation]","[GO:0006412, translation]","[GO:0009399, nitrogen fixation]",None,"[GO:0000917, division septum assembly]","[GO:0052933, alcohol dehydrogenase (cytochrome c(L)) activity]","[GO:0006313, transposition, DNA-mediated]","[GO:0015419, ABC-type sulfate transporter activity]","[GO:0016021, integral component of membrane]","[GO:0032775, DNA methylation on adenine]","[GO:0006313, transposition, DNA-mediated]"
1,"[GO:0006412, translation]","[GO:0005737, cytoplasm]",None,"[GO:0006313, transposition, DNA-mediated]","[GO:0005737, cytoplasm]","[GO:0015049, methane monooxygenase activity]","[GO:0007165, signal transduction]","[GO:0006099, tricarboxylic acid cycle]","[GO:0016612, molybdenum-iron nitrogenase complex]","[GO:0000105, histidine biosynthetic process]","[GO:0022900, electron transport chain]",None,"[GO:0006355, regulation of transcription, DNA-templated]",None,"[GO:0006281, DNA repair]","[GO:0005737, cytoplasm]",None,"[GO:0009252, peptidoglycan biosynthetic process]","[GO:0004803, transposase activity]"
2,"[GO:0006364, rRNA processing]",None,None,"[GO:0003964, RNA-directed DNA polymerase activity]","[GO:0016655, oxidoreductase activity, acting on NAD(P)H, quinone or similar compound as acceptor]",None,"[GO:0000160, phosphorelay signal transduction system]","[GO:0044780, bacterial-type flagellum assembly]","[GO:0018697, carbonyl sulfide nitrogenase activity]","[GO:0065002, intracellular protein transmembrane transport]","[GO:0009055, electron transfer activity]",None,None,None,"[GO:0009432, SOS response]",None,None,"[GO:0008170, N-methyltransferase activity]","[GO:0003677, DNA binding]"
3,"[GO:0005840, ribosome]",None,None,"[GO:0004803, transposase activity]","[GO:0030170, pyridoxal phosphate binding]",None,"[GO:0004888, transmembrane signaling receptor activity]","[GO:0005978, glycogen biosynthetic process]","[GO:0016163, nitrogenase activity]","[GO:0006605, protein targeting]",None,None,None,None,"[GO:0045892, negative regulation of transcription, DNA-templated]",None,None,"[GO:0009007, site-specific DNA-methyltransferase (adenine-specific) activity]",None
4,"[GO:0005737, cytoplasm]",None,None,"[GO:0004802, transketolase activity]","[GO:0006313, transposition, DNA-mediated]",None,None,"[GO:0004601, peroxidase activity]","[GO:0046872, metal ion binding]","[GO:0043952, protein transport by the Sec complex]",None,None,None,None,"[GO:0004803, transposase activity]",None,None,None,None
5,"[GO:0003735, structural constituent of ribosome]",None,None,None,"[GO:0004803, transposase activity]",None,None,"[GO:0051920, peroxiredoxin activity]",None,"[GO:0009099, valine biosynthetic process]",None,None,None,None,"[GO:0003677, DNA binding]",None,None,None,None
6,"[GO:0000049, tRNA binding]",None,None,None,"[GO:0003735, structural constituent of ribosome]",None,None,"[GO:0003677, DNA binding]",None,"[GO:0009097, isoleucine biosynthetic process]",None,None,None,None,None,None,None,None,None
7,"[GO:0005524, ATP binding]",None,None,None,None,None,None,None,None,"[GO:0006457, protein folding]",None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,"[GO:0015986, ATP synthesis coupled proton transport]",None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,"[GO:0006633, fatty acid biosynthetic process]",None,None,None,None,None,None,None,None,None


## BIRCH Enrichment Analysis

In [130]:
df_birch_full_enrichment_data = generate_full_enrichment_data(df_all_merged_birch_data)

df_birch_full_enrichment_data

go-basic.obo: fmt(1.2) rel(2022-01-13) 47,158 GO Terms

Load loci2go Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
100%  3,411 of  3,411 population items found in association

Run loci2go Gene Ontology Analysis: current study set of 25 IDs ...100%     19 of     19 study items found in association
 76%     19 of     25 study items found in population(3411)
Calculating 1,742 uncorrected p-values using fisher_scipy_stats
   1,742 GO terms are associated with  3,407 of  3,411 population items
      22 GO terms are associated with     19 of     25 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run loci2go Gene Ontology Analysis: current study set of 26 IDs ...100%     17 of     17 study items found in association
 65%     17 

   1,742 GO terms are associated with  3,407 of  3,411 population items
      24 GO terms are associated with     24 of     36 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run loci2go Gene Ontology Analysis: current study set of 633 IDs ...100%    516 of    516 study items found in association
 82%    516 of    633 study items found in population(3411)
Calculating 1,742 uncorrected p-values using fisher_scipy_stats
   1,742 GO terms are associated with  3,407 of  3,411 population items
     657 GO terms are associated with    515 of    633 study items
  METHOD fdr_bh:
       3 GO terms found significant (< 0.05=alpha) (  2 enriched +   1 purified): statsmodels fdr_bh
      87 study items associated with significant GO IDs (enriched)
     114 study items associated with 

   1,742 GO terms are associated with  3,407 of  3,411 population items
      26 GO terms are associated with     12 of     18 study items
  METHOD fdr_bh:
       1 GO terms found significant (< 0.05=alpha) (  1 enriched +   0 purified): statsmodels fdr_bh
       4 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run loci2go Gene Ontology Analysis: current study set of 150 IDs ...100%    143 of    143 study items found in association
 95%    143 of    150 study items found in population(3411)
Calculating 1,742 uncorrected p-values using fisher_scipy_stats
   1,742 GO terms are associated with  3,407 of  3,411 population items
     240 GO terms are associated with    143 of    150 study items
  METHOD fdr_bh:
      27 GO terms found significant (< 0.05=alpha) ( 26 enriched +   1 purified): statsmodels fdr_bh
      97 study items associated with significant GO IDs (enriched)
      20 study items associated with 

[[],
 [GOEnrichmentRecord(GO:0009399),
  GOEnrichmentRecord(GO:0022900),
  GOEnrichmentRecord(GO:0016612),
  GOEnrichmentRecord(GO:0018697),
  GOEnrichmentRecord(GO:0016163),
  GOEnrichmentRecord(GO:0046872),
  GOEnrichmentRecord(GO:0009055),
  GOEnrichmentRecord(GO:0051539)],
 [GOEnrichmentRecord(GO:0006313),
  GOEnrichmentRecord(GO:0009432),
  GOEnrichmentRecord(GO:0004803),
  GOEnrichmentRecord(GO:0003677)],
 [],
 [GOEnrichmentRecord(GO:1902358), GOEnrichmentRecord(GO:0015419)],
 [GOEnrichmentRecord(GO:0000160),
  GOEnrichmentRecord(GO:0018106),
  GOEnrichmentRecord(GO:0000155)],
 [],
 [GOEnrichmentRecord(GO:0009252), GOEnrichmentRecord(GO:0008360)],
 [GOEnrichmentRecord(GO:0009236)],
 [GOEnrichmentRecord(GO:0006412),
  GOEnrichmentRecord(GO:0006526),
  GOEnrichmentRecord(GO:0006364),
  GOEnrichmentRecord(GO:0005840),
  GOEnrichmentRecord(GO:0005737),
  GOEnrichmentRecord(GO:0003735),
  GOEnrichmentRecord(GO:0019843),
  GOEnrichmentRecord(GO:0000049),
  GOEnrichmentRecord(GO:0005524

In [133]:
birch_summary_enrichment_results= []
for cluster in df_birch_full_enrichment_data:
    summary_terms = [[go_term.GO, go_term.name] for go_term in cluster]
    birch_summary_enrichment_results.append(summary_terms)
    
df_birch_summary_enrichment_analysis = pd.DataFrame(birch_summary_enrichment_results).T
pickle.dump(df_birch_summary_enrichment_analysis, open('../data/go_terms/df_birch_summary_enrichment_analysis.pickle', 'wb'))

df_birch_summary_enrichment_analysis

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,None,"[GO:0009399, nitrogen fixation]","[GO:0006313, transposition, DNA-mediated]",None,"[GO:1902358, sulfate transmembrane transport]","[GO:0000160, phosphorelay signal transduction system]",None,"[GO:0009252, peptidoglycan biosynthetic process]","[GO:0009236, cobalamin biosynthetic process]","[GO:0006412, translation]",...,"[GO:0003677, DNA binding]",None,"[GO:0000160, phosphorelay signal transduction system]",None,"[GO:0006935, chemotaxis]","[GO:0015031, protein transport]","[GO:0006412, translation]","[GO:0047985, hydrogen dehydrogenase activity]","[GO:0071973, bacterial-type flagellum-dependent cell motility]",None
1,None,"[GO:0022900, electron transport chain]","[GO:0009432, SOS response]",None,"[GO:0015419, ABC-type sulfate transporter activity]","[GO:0018106, peptidyl-histidine phosphorylation]",None,"[GO:0008360, regulation of cell shape]",None,"[GO:0006526, arginine biosynthetic process]",...,None,None,None,None,"[GO:0007165, signal transduction]",None,"[GO:0009102, biotin biosynthetic process]",None,"[GO:0044780, bacterial-type flagellum assembly]",None
2,None,"[GO:0016612, molybdenum-iron nitrogenase complex]","[GO:0004803, transposase activity]",None,None,"[GO:0000155, phosphorelay sensor kinase activity]",None,None,None,"[GO:0006364, rRNA processing]",...,None,None,None,None,None,None,"[GO:0065002, intracellular protein transmembrane transport]",None,"[GO:0009424, bacterial-type flagellum hook]",None
3,None,"[GO:0018697, carbonyl sulfide nitrogenase activity]","[GO:0003677, DNA binding]",None,None,None,None,None,None,"[GO:0005840, ribosome]",...,None,None,None,None,None,None,"[GO:0006605, protein targeting]",None,"[GO:0005576, extracellular region]",None
4,None,"[GO:0016163, nitrogenase activity]",None,None,None,None,None,None,None,"[GO:0005737, cytoplasm]",...,None,None,None,None,None,None,"[GO:0009097, isoleucine biosynthetic process]",None,"[GO:0030694, bacterial-type flagellum basal body, rod]",None
5,None,"[GO:0046872, metal ion binding]",None,None,None,None,None,None,None,"[GO:0003735, structural constituent of ribosome]",...,None,None,None,None,None,None,"[GO:0006814, sodium ion transport]",None,"[GO:0009420, bacterial-type flagellum filament]",None
6,None,"[GO:0009055, electron transfer activity]",None,None,None,None,None,None,None,"[GO:0019843, rRNA binding]",...,None,None,None,None,None,None,"[GO:0015986, ATP synthesis coupled proton transport]",None,"[GO:0005198, structural molecule activity]",None
7,None,"[GO:0051539, 4 iron, 4 sulfur cluster binding]",None,None,None,None,None,None,None,"[GO:0000049, tRNA binding]",...,None,None,None,None,None,None,"[GO:0000105, histidine biosynthetic process]",None,None,None
8,None,None,None,None,None,None,None,None,None,"[GO:0005524, ATP binding]",...,None,None,None,None,None,None,"[GO:0043952, protein transport by the Sec complex]",None,None,None
9,None,None,None,None,None,None,None,None,None,"[GO:0016021, integral component of membrane]",...,None,None,None,None,None,None,"[GO:0009099, valine biosynthetic process]",None,None,None
